In [1]:
import cv2
from skimage.feature import hog
from skimage import exposure
import joblib
import numpy as np

SVM_Linear_model_HOG = joblib.load('SVM_Linear_model_HOG.pkl')
SVM_RBF_model_HOG = joblib.load('SVM_RBF_HOG.pkl')
kNN_model_HOG = joblib.load('kNN_model_HOG.pkl')
RF_model_HOG = joblib.load('RF_model_HOG.pkl')

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Decision

In [2]:
def Predict(img, model):
    img_GRAY = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_GRAY = cv2.resize(img_GRAY, (128, 128))

    hog_feature, hog_image = hog(img_GRAY, orientations=9, pixels_per_cell=(16, 16),
                                 cells_per_block=(3, 3), block_norm='L2-Hys', visualize=True)

    hog_image = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    hog_feature = hog_feature.reshape(1, -1)

    if model == 'SVM_Linear_model_HOG':
        y_pred = SVM_Linear_model_HOG.predict(hog_feature)
    elif model == 'SVM_RBF_model_HOG':
        y_pred = SVM_RBF_model_HOG.predict(hog_feature)
    elif model == 'kNN_model_HOG':
        y_pred = kNN_model_HOG.predict(hog_feature)
    elif model == 'RF_model_HOG':
        y_pred = RF_model_HOG.predict(hog_feature)
    
    return y_pred[0], hog_image

In [16]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import time

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1, detectionCon=0.7)
initial_bounding_box_size = 128
scale_factor = 1.2

phone_number = ''
last_prediction_time = time.time() 
prediction_interval = 3 

while True:
    success, img = cap.read()
    if not success:
        break

    hands, img = detector.findHands(img, draw=False)
    img_copy = img.copy()

    if hands:
        hand = hands[0]
        hand_bbox = hand['bbox'] 

        x, y, w, h = hand_bbox
        bounding_box_size = int(max(w, h) * scale_factor)

        bbox_top_left = (max(0, x - (bounding_box_size - w) // 2), max(0, y - (bounding_box_size - h) // 2))

        bbox_bottom_right = (min(img.shape[1], bbox_top_left[0] + bounding_box_size),
                             min(img.shape[0], bbox_top_left[1] + bounding_box_size))

        cv2.rectangle(img, bbox_top_left, bbox_bottom_right, (0, 255, 0), 2)

        hand_crop = img_copy[bbox_top_left[1]:bbox_bottom_right[1], bbox_top_left[0]:bbox_bottom_right[0]]

        if hand_crop.size != 0:
            imgResized = cv2.resize(hand_crop, (initial_bounding_box_size, initial_bounding_box_size))

            predict, hog_image = Predict(imgResized, model='SVM_Linear_model_HOG')
        
            hog_image = cv2.resize(hog_image, (300, 300))
            imgResized = cv2.resize(imgResized, (300, 300))

            current_time = time.time()
            if current_time - last_prediction_time >= prediction_interval and len(phone_number) < 10:
                phone_number += str(predict)
                last_prediction_time = current_time

            cv2.putText(img, str(predict), (bbox_top_left[0], bbox_top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(img, f"Phone Number: {phone_number}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            time_left = max(0, prediction_interval - (current_time - last_prediction_time))
            cv2.putText(img, f"Time left for next prediction: {time_left:.1f}s", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("ImageCrop", imgResized)
            cv2.imshow("HoG Image", hog_image)
        else:
            print("Error: hand_crop image is empty")

    cv2.imshow("Image", img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        phone_number = ''

cap.release()
cv2.destroyAllWindows()
